In [1]:
import pandas as pd
import warnings

In [2]:
worker_file = "CUPE116NRPPT_TEST2.xlsx"
history_file = 'Payroll_History_TEST2.xlsx'
output_file = 'retro_calc.xlsx'

In [3]:
month_dict = {  "Apr" : ['2022-04-01', '2022-04-30'],
                "May" : ['2022-05-01', '2022-05-31'],
                "Jun" : ['2022-06-01', '2022-06-30'],
                "Jul" : ['2022-07-01', '2022-07-31'],
                "Aug" : ['2022-08-01', '2022-08-31'],
                "Sep" : ['2022-09-01', '2022-09-30'],
                "Oct" : ['2022-10-01', '2022-10-31'],
                "Nov" : ['2022-11-01', '2022-11-30'] }

earning_type_list_1 = ['Arborist Ticket',
                       'Banked Time Off',
                       'Banked Time Payment',
                       'CUPE 116 Evening - pensionable',  # same as evening shift
                       'CUPE 116 Evening Shift',
                       'CUPE 116 Evening Shift 2.0',  # Hours (unprorated)*$1.5
                       'CUPE 116 Night - pensionable',  # same as night shift
                       'CUPE 116 Night Shift',
                       'CUPE 116 Night Shift 2.0',  # same as night shift * 2
                       'CUPE 116 Weekend - pensionable',  # same as weekend shift
                       'CUPE 116 Weekend Premium',
                       'Engineering Technician Certification Premium',
                       'Hourly Pay Staff',
                       'Overtime - Double Time',  # same as Overtime - Double Time Staff
                       'Overtime - Double Time Staff',
                       'Overtime - Time and 1/2',
                       'Overtime meals',
                       'Paid Time Off',  # same as Hourly Pay Staff
                       'Paid Time Off Staff',  # same as Hourly Pay Staff
                       'Regular Earnings Staff',
                       'Statutory Holiday Pay Staff',
                       'Temporary Promotion',
                       'UnPaid Time Off Staff',  # same as Hourly Pay Staff
                       'Vacation Pay $ Hourly Staff',
                       'Vacation Pay $ Staff',
                       'Vacation Pay on Termination Hourly Staff',  #  same as Hourly Pay Staff
                       'Vacation Pay on Termination Staff',  #  same as Hourly Pay Staff
                       'Vacation Payout Hourly',  # same as Hourly Pay Staff
                       'Vacation Payout Salary Staff',  # same as Hourly Pay Staff
                       'WCB Advance 68%']  # same as Hourly Pay Staff

# _2: list of earning types where old rate can be derived from (amount/hours)
earning_type_list_2 = ['Banked Time Off',
                       'Banked Time Payment',
                       'Hourly Pay Staff',
                       'Paid Time Off',
                       'Paid Time Off Staff',
                       'Statutory Holiday Pay Staff',
                       'Vacation Pay $ Hourly Staff',
                       'Vacation Pay $ Staff',
                       'Vacation Pay on Termination Hourly Staff',
                       'Vacation Pay on Termination Staff',
                       'Vacation Payout Hourly',
                       'Vacation Payout Salary Staff']

# _3 depends on hourly rate derived from some earning types in _1
earning_type_list_3 = ['CUPE 116 Grandfathered Eve Shift (5%)',  # Hours (unprorated) * Rate + 5%  i.e. hourly rate is 25.66 * 5% = $1.283
                       'CUPE 116 Grandfathered Night Shift (10%)']  # Hours (unprorated) * Rate + 10%  i.e. hourly rate is 25.66 * 10% = $1.283

# _4 are those that are not applicable
earning_type_list_4 = ['CUPE 116 Aquatic Evening Shift',  # NA - earning not effective until Apr 1, 23
                       'Sub/Head Premiums',  # ???
                       'Vacation Pay Every Pay % Staff']  # ???

# _5 are ones that require special calculations deriving data from earning_type_list_1 iteration
earning_type_list_5 = ['Sub/Head Premiums',
                       'Vacation Pay Every Pay % Staff']



In [4]:
worker_df = pd.read_excel(worker_file, sheet_name=0, header=0, usecols=['emplid','nrppt'])
hist_df = pd.read_excel(history_file, sheet_name=0, header=7, usecols=['Employee ID','Name','Position','Job Category','Pay Group Detail','Period Start Date','Period End Date','Earning','Amount','Hours'])
hist_df.loc[hist_df['Job Category'] == 'CUPE 116']
hist_df['Position'] = hist_df['Position'].str[0:10]

In [5]:
print('Processing...')
temp2_df = pd.DataFrame(columns = ['EMPLID','POS','MONTH','EARNTYPE','SUMHRS','SUMAMT','RETRO','NOTES'])
no_hist_list = []
for index, row in worker_df.iterrows():
    vemplid = row['emplid']
    vnrppt = row['nrppt']
    print(vemplid, vnrppt)
    temp1_df = hist_df.loc[hist_df['Employee ID'] == vemplid]
    temp1_df = temp1_df.loc[(temp1_df['Period End Date'] < vnrppt)]
    #
    pos_arr = temp1_df.Position.unique()
    if pos_arr.size == 0:
        no_hist_list.append(vemplid)
    else:
        work_list = []
        for vpos in pos_arr:
            # Determine if Salaried or Hourly
            paygroup_arr = hist_df.loc[hist_df['Position'] == vpos]['Pay Group Detail'].str[0:6].unique()
            if 'Salary' in paygroup_arr:
                vpaygroup = 'Salary'
            else:
                vpaygroup = 'Hourly'
            #
            # Old rate determination:
            # For Hourly:  Collect the rates per month (amount / hours) for 'Hourly Pay Staff'.  Use the highest.
            # For Salary:  Collect the rates per month of earning types in earning_type_list_2 first.  If none, get rate using 'Regular Earnings Staff'.
            for vmonth, vdate in month_dict.items():
                voldrate_list = []  # For salary, this would be for earns in earning_type_list_2.  For hourly: only Hourly Pay Staff.
                voldrate_reg = 0  # For salary this would be 'Regular Earnings Staff'.  For hourly, this would be 'Hourly Pay Staff'

                # Accumulators for earning_type_list_5.  Number denotes index on list.

                vaccum_reg_amt = 0
                vaccum_hourlypay_amt = 0
                vaccum_statholpaystaff_amt = 0
                vaccum_paidtimeoffstaff_amt = 0
                vaccum_wcbadv_amt = 0

                vaccum_reg_retro = 0
                vaccum_hourlypay_retro = 0
                vaccum_statholpaystaff_retro = 0
                vaccum_paidtimeoffstaff_retro = 0
                vaccum_wcbadv_retro = 0

                # End accumulators
                
                # We need to start iterating _1 first to derive old rate (for both hourly and salaried)
                for vearn in earning_type_list_1:
                    vnotes = ''
                    #
                    temp_df = temp1_df.loc[(temp1_df['Position']==vpos) & (temp1_df['Earning']==vearn) & (temp1_df['Period End Date'] >= vdate[0]) & (temp1_df['Period End Date'] <= vdate[1])]
                    vhrs_sum = temp_df['Hours'].sum()
                    vamt_sum = temp_df['Amount'].sum()
                    # Determine old rate
                    if vpaygroup == 'Hourly':
                        vnotes = 'H'
                        if vearn == 'Hourly Pay Staff' and vhrs_sum != 0:
                            voldrate_reg = (vamt_sum/vhrs_sum)
                            vnotes = vnotes + '|' + str(round(vamt_sum/vhrs_sum, 2))
                    if vpaygroup == 'Salary':
                        vnotes = 'S'
                        if vearn in earning_type_list_2 and vhrs_sum != 0:
                            voldrate_list.append(vamt_sum/vhrs_sum)
                            vnotes = vnotes + '|' + str(round(vamt_sum/vhrs_sum, 2))
                        if vearn == 'Regular Earnings Staff' and vhrs_sum != 0:
                            voldrate_reg = (vamt_sum/vhrs_sum)
                            vnotes = vnotes + '|' + str(round(vamt_sum/vhrs_sum, 2))
                    # Start retro calculation
                    vretro = 0
                    if vearn in ['Arborist Ticket']:
                        vretro = vhrs_sum * 0.25
                    elif vearn in ['CUPE 116 Evening Shift 2.0']:
                        vretro = vhrs_sum * 1.5
                    elif vearn in ['CUPE 116 Evening - pensionable','CUPE 116 Evening Shift']:
                        vretro = vhrs_sum * 0.2
                    elif vearn in ['CUPE 116 Night - pensionable','CUPE 116 Night Shift']:
                        vretro = vhrs_sum * 0.25
                    elif vearn in ['CUPE 116 Night Shift 2.0']:
                        vretro = vhrs_sum * 0.25 * 2
                    elif vearn in ['CUPE 116 Weekend - pensionable','CUPE 116 Weekend Premium']:
                        vretro = vhrs_sum * 0.2
                    elif vearn in ['Engineering Technician Certification Premium']:
                        vretro = vhrs_sum * 0.25
                    elif vearn in ['Overtime meals']:
                        vretro = vhrs_sum * 8
                    else:
                        if vhrs_sum != 0:
                            # GWI retro on hours
                            #
                            # Banked Time Off
                            # Banked Time Payment
                            # Hourly Pay Staff
                            # Overtime - Double Time
                            # Overtime - Double Time Staff
                            # Overtime - Time and 1/2
                            # Paid Time Off
                            # Paid Time Off Staff
                            # Regular Earnings Staff
                            # Statutory Holiday Pay Staff
                            # Temporary Promotion
                            # UnPaid Time Off Staff
                            # Vacation Pay $ Hourly Staff
                            # Vacation Pay $ Staff
                            # Vacation Pay on Termination Hourly Staff
                            # Vacation Pay on Termination Staff
                            # Vacation Payout Hourly
                            # Vacation Payout Salary Staff
                            # WCB Advance 68%
                            #
                            vretro = (((((vamt_sum/vhrs_sum) + 0.25) * 1.0324) - (vamt_sum/vhrs_sum))) * vhrs_sum
                        else:
                            vretro = 0
                    # End retro calculation

                    if vretro != 0:

                        work_list.append([vemplid, vpos, vmonth, vearn, vhrs_sum, vamt_sum, vretro, vnotes])
                        
                        if vpaygroup == 'Salary' and vearn == 'Regular Earnings Staff':
                            vaccum_reg_amt = vaccum_reg_amt + vamt_sum
                            vaccum_reg_retro = vaccum_reg_retro + vretro
                        if vearn == 'Hourly Pay Staff':
                            vaccum_hourlypay_amt = vaccum_hourlypay_amt + vamt_sum
                            vaccum_hourlypay_retro = vaccum_hourlypay_retro + vretro
                        if vearn == 'Statutory Holiday Pay Staff':
                            vaccum_statholpaystaff_amt = vaccum_statholpaystaff_amt + vamt_sum
                            vaccum_statholpaystaff_retro = vaccum_statholpaystaff_retro + vretro
                        if vearn == 'Paid Time Off Staff':
                            vaccum_paidtimeoffstaff_amt = vaccum_paidtimeoffstaff_amt + vamt_sum
                            vaccum_paidtimeoffstaff_retro = vaccum_paidtimeoffstaff_retro + vretro
                        if vearn == 'WCB Advance 68%':
                            vaccum_wcbadv_amt = vaccum_wcbadv_amt + vamt_sum
                            vaccum_wcbadv_retro = vaccum_wcbadv_retro + vretro

                # end for vearn 1

                # Pick BEST OLD RATE for the month
                if vpaygroup == 'Hourly':
                    voldrate = voldrate_reg
                if vpaygroup == 'Salary':
                    voldrate = 0
                    if len(voldrate_list) > 0:
                        voldrate = max(voldrate_list)
                    elif voldrate_reg > 0:
                        voldrate = voldrate_reg
                vnotes = str(round(voldrate,2))
            
                # work_list.append([vemplid, vpos, '', 'BEST OLD RATE', 0, 0, 0, vnotes])

                for vearn in earning_type_list_3:
                    vnotes = ''
                    if vpaygroup == 'Hourly':
                        vnotes = 'H'
                    elif vpaygroup == 'Salary':
                        vnotes = 'S'
                    temp_df = temp1_df.loc[(temp1_df['Position']==vpos) & (temp1_df['Earning']==vearn) & (temp1_df['Period End Date'] >= vdate[0]) & (temp1_df['Period End Date'] <= vdate[1])]
                    vhrs_sum = temp_df['Hours'].sum()
                    vamt_sum = temp_df['Amount'].sum()
                    # Start retro calculation
                    vretro = 0
                    if vhrs_sum != 0:
                        if vearn in ['CUPE 116 Grandfathered Eve Shift (5%)']:
                            vretro = ( ( (((voldrate + 0.25) * 1.0324) * 1.05) - ((voldrate + 0.25) * 1.0324) )  -  ( (voldrate * 1.05) - voldrate ) ) * vhrs_sum
                        elif vearn in ['CUPE 116 Grandfathered Night Shift (10%)']:
                            vretro = ( ( (((voldrate + 0.25) * 1.0324) * 1.10) - ((voldrate + 0.25) * 1.0324) )  -  ( (voldrate * 1.10) - voldrate ) ) * vhrs_sum
                    # End retro calculation
                    
                    if vretro != 0:
                        work_list.append([vemplid, vpos, vmonth, vearn, vhrs_sum, vamt_sum, vretro, vnotes])

                # end for vearn 3

                for vearn in earning_type_list_5:
                    vnotes = ''
                    if vpaygroup == 'Hourly':
                        vnotes = 'H'
                    elif vpaygroup == 'Salary':
                        vnotes = 'S'
                    temp_df = temp1_df.loc[(temp1_df['Position']==vpos) & (temp1_df['Earning']==vearn) & (temp1_df['Period End Date'] >= vdate[0]) & (temp1_df['Period End Date'] <= vdate[1])]
                    vhrs_sum = temp_df['Hours'].sum()
                    vamt_sum = temp_df['Amount'].sum()
                    # Start retro calculation
                    vretro = 0
                    if vearn in ['Sub/Head Premiums']:
                        if vaccum_reg_amt != 0:
                            vretro = (vamt_sum / vaccum_reg_amt) * vaccum_reg_retro
                    elif vearn in ['Vacation Pay Every Pay % Staff']:
                        if (vaccum_hourlypay_amt + vaccum_statholpaystaff_amt + vaccum_paidtimeoffstaff_amt + vaccum_wcbadv_amt) != 0: 
                            vretro = (vamt_sum / (vaccum_hourlypay_amt + vaccum_statholpaystaff_amt + vaccum_paidtimeoffstaff_amt + vaccum_wcbadv_amt)) * (vaccum_hourlypay_retro + vaccum_statholpaystaff_retro + vaccum_paidtimeoffstaff_retro + vaccum_wcbadv_retro)
                    # End retro calculation
                    
                    if vretro != 0:
                        work_list.append([vemplid, vpos, vmonth, vearn, vhrs_sum, vamt_sum, vretro, vnotes])

                # end for vearn 5

            # end for vmonth

        # end for vpos
            
        temp_df = pd.DataFrame(work_list, columns=['EMPLID','POS','MONTH','EARNTYPE','SUMHRS','SUMAMT','RETRO','NOTES'])
        temp2_df = temp2_df.append(temp_df)
        temp_df.drop(['MONTH','SUMHRS','SUMAMT','NOTES'], axis=1, inplace=True)
        temp_group = temp_df.groupby(['EMPLID','POS','EARNTYPE'])
        temp_df = temp_group.sum()        

temp_df = temp2_df.drop(['MONTH','SUMHRS','SUMAMT','NOTES'], axis=1, inplace=False)
temp3_df = temp_df.groupby(['EMPLID','POS','EARNTYPE']).sum('RETRO').reset_index()
temp3_df = temp3_df[temp3_df.EARNTYPE != 'BEST OLD RATE']

no_hist_df = pd.DataFrame(no_hist_list, columns=['EMPLID'])

Processing...
3626865 2022-11-16 00:00:00
3737128 2022-11-16 00:00:00
3857328 2022-11-16 00:00:00
3900126 2022-11-16 00:00:00
4050886 2022-11-16 00:00:00
4054946 2022-11-16 00:00:00
4055012 2022-11-16 00:00:00
4061829 2022-11-16 00:00:00
4268202 2022-11-16 00:00:00
4372638 2022-11-16 00:00:00
4483278 2022-11-16 00:00:00
4496361 2022-11-16 00:00:00
4776712 2022-11-16 00:00:00
5224845 2022-11-16 00:00:00
5234611 2022-11-16 00:00:00
5301351 2022-11-16 00:00:00
5361494 2022-11-16 00:00:00
5525691 2022-11-16 00:00:00
5636841 2022-11-16 00:00:00
5661331 2022-11-16 00:00:00
5710383 2022-11-16 00:00:00
5807794 2022-11-16 00:00:00
5834376 2022-11-16 00:00:00
1284754 2022-11-24 00:00:00
5825245 2022-11-24 00:00:00
1822888 2022-04-01 00:00:00
2975793 2022-04-01 00:00:00
5717469 2022-04-01 00:00:00
3542475 2022-04-09 00:00:00
2610353 2022-04-16 00:00:00
3710165 2022-04-16 00:00:00
4505344 2022-04-16 00:00:00
5735246 2022-04-16 00:00:00
1307592 2022-04-24 00:00:00
2719444 2022-04-24 00:00:00
348608

In [6]:
def prep_sheet(df1, sheet1):
    column_widths = {
    "EMPLID": 50,
    "POS": 20,
    "MONTH": 10,
    "EARNTYPE": 50,
    "SUMHRS": 10,
    "SUMAMT": 10,
    "RETRO": 10,
    "NOTES": 50 }
    df1.to_excel(writer, sheet_name = sheet1, index=False, header=False, startrow=1)
    for column in df1:
        column_width = column_widths[column]
        col_idx = df1.columns.get_loc(column)
        writer.sheets[sheet1].set_column(col_idx, col_idx, column_width)
    column_settings1 = [{'header': column} for column in df1.columns]
    (max_row, max_col) = df1.shape
    worksheet1 = writer.sheets[sheet1]
    format1 = workbook1.add_format({
        'align': 'left', 
        'valign': 'top', 
        'text_wrap': False
    })
    worksheet1.set_column('A:Z', None, format1)
    worksheet1.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings1, 'style': None})
    worksheet1.freeze_panes(1, 0)

In [7]:
writer = pd.ExcelWriter(output_file, engine = 'xlsxwriter')
workbook1 = writer.book

temp2_df.to_excel(writer, sheet_name = 'Period Retros', index=False, header=False, startrow=1)
prep_sheet(temp2_df, 'Period Retros')
temp3_df.to_excel(writer, sheet_name = 'Retro Totals', index=False, header=False, startrow=1)
prep_sheet(temp3_df, 'Retro Totals')
if no_hist_df.empty is False:
    no_hist_df.to_excel(writer, sheet_name = 'No History', index=False, header=False, startrow=1)
    prep_sheet(no_hist_df, 'No History')

writer.save()

In [8]:
print('Done')

Done
